In [1]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="sk-0rLvuRkMiD4Mw25QYygh6rUlZVjpQWNGNF4yez7z3PZ7yCOm",
    # api_key="sk-tMbkq3K1iO5vf0FRMlrmzslGXJZwE0us3mve4QXuvpnZcumG",
    base_url="https://api.chatanywhere.tech/v1"
    # base_url="https://api.chatanywhere.cn/v1"
)

In [2]:
def gpt_35_api(messages: list):

    completion = client.chat.completions.create(model="gpt-3.5-turbo-ca", messages=messages, temperature=0.5)
    return completion.choices[0].message.content

In [3]:
import json
from tqdm import tqdm

In [4]:
from langchain import hub
prompt = hub.pull("tyfann/llm4commit-rag-4sim")

In [5]:
with open('../data/scenario/multi_diff/multi_diff_rag_top4_django_prompt.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

In [6]:
gpt_msg = []
for data in tqdm(org_data[:100], total=len(org_data[:100]), desc="Processing documents"):
    messages = prompt.invoke(
        {"diff1": data['sim_diff_0'],  "msg1": data['sim_msg_0'],
         "diff2": data['sim_diff_1'],  "msg2": data['sim_msg_1'],
         "diff3": data['sim_diff_2'],  "msg3": data['sim_msg_2'],
         "diff4": data['sim_diff_3'],  "msg4": data['sim_msg_3'],
         "question": data['org_diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    gpt_msg.append(gpt_35_api(example_prompt))

In [7]:
with open('../data/scenario/filtered_data/project_based_django_diff_filtered.json', 'r', encoding='UTF-8') as f:
    chronicle_data = json.load(f)

for item, msg in zip(chronicle_data[:100], gpt_msg):
    item['chatgpt_rag'] = msg

output_file = '../data/scenario/top4/scenario_django_chatgpt_rag_top4.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(chronicle_data[:100], f, ensure_ascii=False, indent=4)

In [6]:
with open('../data/scenario/multi_diff/multi_diff_rag_top4_django_prompt.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

gpt_msg = []
for index, data in tqdm(enumerate(org_data[:100]), total=len(org_data[:100]), desc="Processing documents"):
    messages = prompt.invoke(
        {"diff1": data['sim_diff_0'],  "msg1": data['sim_msg_0'],
         "diff2": data['sim_diff_1'],  "msg2": data['sim_msg_1'],
         "diff3": data['sim_diff_2'],  "msg3": data['sim_msg_2'],
         "diff4": data['sim_diff_3'],  "msg4": data['sim_msg_3'],
         "question": data['org_diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    try:
        gpt_msg.append(gpt_35_api(example_prompt))
    except:
        print(index)
        rag_prompt = hub.pull("tyfann/llm4commit-rag")
        messages = rag_prompt.invoke(
            {"context": data['sim_diff_0'], "msg": data['sim_msg_0'], "diff": data['org_diff']}
        ).to_messages()
        example_prompt = [{'role': 'user','content': messages[0].content},]
        gpt_msg.append(gpt_35_api(example_prompt))

with open('../data/scenario/filtered_data/project_based_django_multi_diff_filtered.json', 'r', encoding='UTF-8') as f:
    chronicle_data = json.load(f)

for item, msg in zip(chronicle_data[:100], gpt_msg):
    item['chatgpt_rag'] = msg

output_file = '../data/scenario/multi_diff/multi_diff_rag_top4_django.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(chronicle_data[:100], f, ensure_ascii=False, indent=4)

In [7]:
with open('../data/scenario/multi_diff/multi_diff_rag_top4_react_prompt.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

gpt_msg = []
for index, data in tqdm(enumerate(org_data[:100]), total=len(org_data[:100]), desc="Processing documents"):
    messages = prompt.invoke(
        {"diff1": data['sim_diff_0'],  "msg1": data['sim_msg_0'],
         "diff2": data['sim_diff_1'],  "msg2": data['sim_msg_1'],
         "diff3": data['sim_diff_2'],  "msg3": data['sim_msg_2'],
         "diff4": data['sim_diff_3'],  "msg4": data['sim_msg_3'],
         "question": data['org_diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    try:
        gpt_msg.append(gpt_35_api(example_prompt))
    except:
        print(index)
        rag_prompt = hub.pull("tyfann/llm4commit-rag")
        messages = rag_prompt.invoke(
            {"context": data['sim_diff_0'], "msg": data['sim_msg_0'], "diff": data['org_diff']}
        ).to_messages()
        example_prompt = [{'role': 'user','content': messages[0].content},]
        gpt_msg.append(gpt_35_api(example_prompt))

with open('../data/scenario/filtered_data/project_based_react_multi_diff_filtered.json', 'r', encoding='UTF-8') as f:
    chronicle_data = json.load(f)

for item, msg in zip(chronicle_data[:100], gpt_msg):
    item['chatgpt_rag'] = msg

output_file = '../data/scenario/multi_diff/multi_diff_rag_top4_react_v2.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(chronicle_data[:100], f, ensure_ascii=False, indent=4)

In [8]:
with open('../data/scenario/multi_diff/multi_diff_rag_top4_flask_prompt.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

gpt_msg = []
for index, data in tqdm(enumerate(org_data[:100]), total=len(org_data[:100]), desc="Processing documents"):
    messages = prompt.invoke(
        {"diff1": data['sim_diff_0'],  "msg1": data['sim_msg_0'],
         "diff2": data['sim_diff_1'],  "msg2": data['sim_msg_1'],
         "diff3": data['sim_diff_2'],  "msg3": data['sim_msg_2'],
         "diff4": data['sim_diff_3'],  "msg4": data['sim_msg_3'],
         "question": data['org_diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    try:
        gpt_msg.append(gpt_35_api(example_prompt))
    except:
        print(index)
        rag_prompt = hub.pull("tyfann/llm4commit-rag")
        messages = rag_prompt.invoke(
            {"context": data['sim_diff_0'], "msg": data['sim_msg_0'], "diff": data['org_diff']}
        ).to_messages()
        example_prompt = [{'role': 'user','content': messages[0].content},]
        gpt_msg.append(gpt_35_api(example_prompt))

with open('../data/scenario/filtered_data/project_based_flask_multi_diff_filtered.json', 'r', encoding='UTF-8') as f:
    chronicle_data = json.load(f)

for item, msg in zip(chronicle_data[:100], gpt_msg):
    item['chatgpt_rag'] = msg

output_file = '../data/scenario/multi_diff/multi_diff_rag_top4_flask_v2.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(chronicle_data[:100], f, ensure_ascii=False, indent=4)

In [17]:
data = org_data[11]
messages = prompt.invoke(
        {"diff1": data['sim_diff_0'],  "msg1": data['sim_msg_0'],
         "diff2": data['sim_diff_1'],  "msg2": data['sim_msg_1'],
         "diff3": data['sim_diff_2'],  "msg3": data['sim_msg_2'],
         "diff4": data['sim_diff_3'],  "msg4": data['sim_msg_3'],
         "question": data['org_diff']}
).to_messages()
example_prompt = [{'role': 'user','content': messages[0].content},]

In [18]:
gpt_35_api(example_prompt)

In [9]:
import json

# 打开 JSON 文件
with  open('../data/chronicle/rag_dev/codebert/chronicle_rag_dev_codebert.json', 'r+') as f:
    # 读取 JSON 数据
    data = json.load(f)
    # 保留前 100 条数据
    data = data[:100]
    # 将文件指针移到文件开头
    f.seek(0)
    # 将数据写回文件
    json.dump(data, f, indent=4)
    # 删除文件末尾的空白字符
    f.truncate()